# Обучение продвинутой RNN для предсказания замаскированных слов
## Подготовка данных

In [5]:
!pip install datasets transformers


import torch
import torch.nn as nn
import re
import random

from datasets import load_dataset
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizerFast
from sklearn.model_selection import train_test_split

random.seed(42)
torch.manual_seed(42)

# Очистка текста
def clean_string(text):
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text


   ---------------------------------------- 0.0/26.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/26.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/26.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/26.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/26.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/26.2 MB ? eta -:--:--
   ---------------------------------------- 0.3/26.2 MB ? eta -:--:--
   ---------------------------------------- 0.3/26.2 MB ? eta -:--:--
   ---------------------------------------- 0.3/26.2 MB ? eta -:--:--
   ---------------------------------------- 0.3/26.2 MB ? eta -:--:--
    --------------------------------------- 0.5/26.2 MB 335.7 kB/s eta 0:01:17
    --------------------------------------- 0.5/26.2 MB 335.7 kB/s eta 0:01:17
    --------------------------------------- 0.5/26.2 MB 335.7 kB/s eta 0:01:17
    --------------------------------------- 0.5/26.2 MB 335.7 k

d:\first_sprint\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="train") # Загрузили обучающую часть датасета в СЫРОМ виде

seq_len = 7 # Зададим длину окна для контекста (тут будет 3 токена до маски, маска и 3 токена после)

texts = [line for line in dataset["text"] if len(line.split()) >= seq_len] # Оставим строчки гдне не меньше чем 7 слов, иначе не сможем построить контекст (т.к. задали 7 токенов)

cleaned_texts = list(map(clean_string, texts)) # Очищаем с помощью нашей функции

max_texts_count = 7000 # Используем только 7к строк, чтобы обучение было недолгое + цпу выдержал

train_texts, val_texts = train_test_split(cleaned_texts[:max_texts_count], test_size=0.05, random_state=42)

print(f"Train texts: {len(train_texts)}, Val texts: {len(val_texts)}")


d:\first_sprint\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\User\.cache\huggingface\hub\datasets--wikitext. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Generating validation split: 100%|██████████| 3760/3760 [00:00<00:00, 341406.34 examples/s]


Train texts: 6650, Val texts: 350


Загрузил датасет отфильтровали слишком короткие строки, почистили текст, ограничили объём (7000 строк) и разделили его на обучающую и валидационную выборки

In [ ]:
class MaskedBertDataset(Dataset): # Архитектура датасета
    def __init__(self, texts, tokenizer, seq_len=7): # текст, токенизатор (берт), длина окна
        self.samples = []
        for line in texts:
            token_ids = tokenizer.encode(line, add_special_tokens=False, max_length=512, truncation=True) # токенизация, важно что без sep и сls, т.к. задача другая
            if len(token_ids) < seq_len: # Пропуск мелких токенов
                continue
            for i in range(1, len(token_ids) - 1):
                context = token_ids[max(0, i - seq_len//2): i] + [tokenizer.mask_token_id] + token_ids[i+1: i+1+seq_len//2] 
                # самая важная часть, береем seq_len//2 до текущего token_ids[i - 3 : i], вставляем маску, дабавляем seq_len // 2 токенов после token_ids[i+1 : i+4]
                if len(context) < seq_len:
                    continue
                target = token_ids[i] # Хапомниаем замаскированный токен (y) - наш таргет 
                self.samples.append((context, target))

    def __len__(self): # Вернем количество пар X y в датасете 
        return len(self.samples) 

    def __getitem__(self, idx): # обращение к датасету по индексу 
        x, y = self.samples[idx]
        return torch.tensor(x), torch.tensor(y)


Создали класс MaskedBertDataset:
- получает строки,
- превращает их в токены,
- выбирает каждый токен как потенциальный "пропущенный",
- формирует контекст вокруг него,
- возвращает (x, y):

x — токены с <MASK> в центре
y — правильный токен, который заменили

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased") # Используем берт токенизатор, потому что он подходит под задачу и знает как разбивать на маски

train_dataset = MaskedBertDataset(train_texts, tokenizer, seq_len=seq_len)
val_dataset = MaskedBertDataset(val_texts, tokenizer, seq_len=seq_len)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True) # Бьем нашу выборку на батчи по 64, перемешиваем данные перед эпохой, доджим переобучение
val_loader = DataLoader(val_dataset, batch_size=64) # Валидационную выборку не мешаем, так стабильнее результаты

print("Train dataset size:", len(train_dataset))
print("Val dataset size:", len(val_dataset))
print("Train loader size:", len(train_loader))
print("Val loader size:", len(val_loader))


Train dataset size: 592449
Val dataset size: 29617
Train loader size: 9258
Val loader size: 463


In [14]:
# Возьмем первый батч и посмотрим как выглядит
for x_batch, y_batch in train_loader:
    print("x_batch shape:", x_batch.shape)
    print("y_batch shape:", y_batch.shape)

    print("\nПример токенов x[0]:", x_batch[0].tolist())
    print("Таргет для x[0]:", y_batch[0].item())

    decoded = tokenizer.decode(x_batch[0], skip_special_tokens=True)
    print("\nПример декодированного x[0]:", decoded)

    true_token = tokenizer.decode([y_batch[0].item()])
    print("Замаскированный токен:", true_token)
    
    break

x_batch shape: torch.Size([64, 7])
y_batch shape: torch.Size([64])

Пример токенов x[0]: [8213, 2020, 2583, 103, 22806, 2000, 3088]
Таргет для x[0]: 2000

Пример декодированного x[0]: generations were able migrate to africa
Замаскированный токен: to


На этом этапе мы подготовили данные для обучения модели, которая будет предсказывать замаскированное слово в предложении по контексту вокруг него

1. Загрузили датасет WikiText-2, содержащий тексты с Википедии
2. Очистили тексты от лишних символов и привели их к нижнему регистру
3. Отобрали только те строки, где хотя бы 7 слов (нам нужен контекст: 3 токена до, `<MASK>`, 3 токена после).
4. Разбили данные на обучающую и валидационную выборки.
5. Реализовали класс `MaskedBertDataset`, который:
   - Токенизирует предложение.
   - Маскирует по одному токену в каждом окне.
   - Сохраняет пары: вход (контекст с `<MASK>`) и правильный токен
6. Создали `DataLoader`'ы для обучения и валидации

- Обучающих примеров: **592449**
- Валидационных примеров: **29617**
- Батчей в обучении: **9258**
- Батчей в валидации: **463**

Теперь у нас есть полноценная выборка для обучения языковой модели, способной угадывать слова по контексту